In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [4]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"inputs":input}, {"output":output})
    
add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

In [15]:
memory.load_memory_variables({})
add_message(5, 5) # save recent messages (k = 4)
memory.load_memory_variables({})

{'history': [HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [5]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)

def get_history():
    return memory.load_memory_variables({})

In [ ]:
add_message("Hi I'm Noguri, I live in South Korea", "Wow that is so cool!")

In [ ]:
add_message("South Korea is so pretty")

In [6]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True
)

In [21]:
add_message("Hi I'm Noguri, I live in South Korea", "Wow that is so cool!")

In [ ]:
add_message("South Korea is so pretty")

In [ ]:
get_history()

In [25]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

# 중요한 요약본을 생성하여 메모리에 저장
memory = ConversationKGMemory(
    llm = llm,
    return_messages=True,
)


In [10]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
    
)

template = """
You are a helpful AI talking to a human.

{chat_history}
Human:{question}
You:
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [11]:
invoke_chain("My name is noguri")

KeyError: 'history'